# Data Cleaning

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [1]:
import unicodedata
from nltk.stem import PorterStemmer, WordNetLemmatizer


In [2]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd

data_df = pd.read_csv('Combined_NewsAPI_data.csv')

In [3]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [6]:
data_df = data_df.astype({'Description':'string'})

In [10]:
data_df = data_df.dropna(axis = 0, how ='any')

In [11]:
data_df[data_df.Description.isnull()]

,Description,Label


In [12]:
data_df['Description'] = data_df['Description'].apply(round1)

/var/folders/px/bp9nx6057cv6t7g9910khmkr0000gn/T/ipykernel_49319/1810766649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Description'] = data_df['Description'].apply(round1)


In [13]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    # text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8')
    # text = unicodedata.normalize('NFKD', unicode(text, 'utf8'))

    return text

round2 = lambda x: clean_text_round2(x)

In [14]:
# Let's take a look at the updated text
data_df['Description'] = data_df['Description'].apply(round2)


/var/folders/px/bp9nx6057cv6t7g9910khmkr0000gn/T/ipykernel_49319/3413303479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Description'] = data_df['Description'].apply(round2)


In [15]:
# Stemming

porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

data_df['Description'] = data_df['Description'].apply(stem_sentences)



/var/folders/px/bp9nx6057cv6t7g9910khmkr0000gn/T/ipykernel_49319/1183852345.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Description'] = data_df['Description'].apply(stem_sentences)


**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [16]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_df['Description'])
data_countVec = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_countVec.index = data_df.index
data_countVec['Label'] = data_df['Label']
data_countVec

,abandon,abil,abl,abroad,abu,abus,acc,acceler,access,accompani,...,zambia,zelenskyy,zero,zerowast,zesco,zombi,zone,zoo,zoochest,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,renewable energy
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,renewable energy
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,renewable energy
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,renewable energy
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,renewable energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,global warming
491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,global warming
492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,global warming
493,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,global warming


In [68]:
#  tempdf = data_countVec[data_countVec['Labels'] == 'climate_disruption']

In [69]:
#  tempdf =tempdf.sum(axis=0,numeric_only=True)

In [17]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(smooth_idf=True)
feature_matrix = tfidf.fit_transform(data_df['Description'])
# data_countVec = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_tfidf = pd.DataFrame(data=feature_matrix.todense(), columns=tfidf.get_feature_names_out())
data_tfidf['Label'] = data_df['Label']
data_tfidf

,abandon,abil,abl,about,abroad,abu,abus,acc,acceler,access,...,zambia,zelenskyy,zero,zerowast,zesco,zombi,zone,zoo,zoochest,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,renewable energy
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,renewable energy
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,renewable energy
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,renewable energy
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,renewable energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,global warming
490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,global warming
491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,global warming
492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,global warming


In [19]:
from sklearn.feature_extraction.text import TfidfTransformer 
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(data_df['Description'])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

              TF-IDF
photovolta  0.303375
swath       0.303375
asphalt     0.303375
urban       0.284484
closer      0.271080
where       0.260684
franc       0.260684
cover       0.260684
area        0.252189
bring       0.245007
even        0.209498
need        0.201003
plan        0.185698
renew       0.161527
will        0.138397
energi      0.135138
to          0.134124
with        0.132685
it          0.108174
in          0.083534
of          0.073639
precis      0.000000
play        0.000000
plc         0.000000
pleas       0.000000
